# Postprocessing and visuals

In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import seaborn as sns

from dosipy.utils.viz import fig_config, save_fig

In [ ]:
# features
h = [2, 5, 10, 50, 150]
f = [6, 26, 60, 90]
A = [4, 4, 1, 1]
r = [5, 7, 10, 15]
columns=['f', 'd', 'r', 'sPDn_pln', 'sPDn_sph', 'sPDn_cyl']

# load data
sPDn_at6GHz = np.load(os.path.join('data', 'sPDn_at6GHz.npy'))
sPDn_at26GHz = np.load(os.path.join('data', 'sPDn_at26GHz.npy'))
sPDn_at60GHz = np.load(os.path.join('data', 'sPDn_at60GHz.npy'))
sPDn_at90GHz = np.load(os.path.join('data', 'sPDn_at60GHz.npy'))

# populate and engineer dataset
def rpd(x, x_ref):
    """Return relative percentage difference between data and a given
    reference value."""
    return np.abs(x - x_ref) / x_ref * 100

rep = sPDn_at6GHz.shape[0]
data = np.vstack((sPDn_at6GHz, sPDn_at26GHz, sPDn_at60GHz, sPDn_at90GHz))
dataset = pd.DataFrame(np.hstack((np.repeat(f, rep).reshape(-1, 1), data)),
                       columns=columns)
dataset['rpd_sph'] = rpd(dataset['sPDn_sph'], dataset['sPDn_pln'])
dataset['rpd_cyl'] = rpd(dataset['sPDn_cyl'], dataset['sPDn_pln'])
dataset['f'] = np.asarray(dataset['f'], 'int')
dataset['d'] = np.asarray(dataset['d'] * -1000, 'int')
dataset['r'] = np.asarray(dataset['r'] * 100, dtype='int')

# plotting utils
nrows = 2
ncols = 2
markerkwargs = {'lw': 2,
                'mec': 'k',
                'mfc': 'white',
                'mfcalt': 'white',
                'mew': 1,
                'ms': 10}

## Results - incident power densities for all scenarios

### Incident power density as a function of the seperation distance

In [ ]:
fig_config(latex=True, text_size=18)
fig = plt.figure(figsize=(9, 7.5))
gs = gridspec.GridSpec(2, 2, figure=fig)
for i, _f in enumerate(f):
    ghost = fig.add_subplot(gs[i])
    ghost.set_title(f'$f =$ {_f} GHz\n')
    ghost.axis('off')
    gs_in = gs[i].subgridspec(2, 2)
    for i_in, _r in enumerate(r):
        ax = fig.add_subplot(gs_in[i_in])
        ax.plot(h, dataset[(dataset['f']==_f) & (dataset['r']==_r)]['sPDn_sph'],
                'ko-', label='spherical', **markerkwargs)
        ax.plot(h, dataset[(dataset['f']==_f) & (dataset['r']==_r)]['sPDn_cyl'],
                'kd--', dashes=(3, 1.5), label='cylndrical', **markerkwargs)
        ax.plot(h, dataset[(dataset['f']==_f) & (dataset['r']==_r)]['sPDn_pln'],
                'ks:', label='planar', zorder=0, **markerkwargs)
        ax.set(title=f'$r =$ {_r} cm', xscale='log')
        if (_f in [60, 90]) & (_r in [10, 15]):
            ax.set(xlabel='', xticks=h, xticklabels=h)
        else:
            ax.set(xlabel='', xticks=h, xticklabels=[])
        if (_f in [6, 60]) & (_r in [5, 10]):
            ax.set(ylabel='', yticks=[0, 15, 30])
        else:
            ax.set(ylabel='', yticks=[0, 15, 30], yticklabels=[])

handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.225, 0.6125))
fig.supxlabel('$d$ [mm]')
fig.supylabel('$sPD_{\\text{n}}$ [W/m$^2$]')
fig.tight_layout()
fname = os.path.join('figures', 'sPDn_abs')
save_fig(fig, fname=fname)

### RPD between incident power density on the spherical/cylindrical and planar surface as a function of the seperation distance

In [ ]:
fig_config(latex=True, text_size=18)
fig = plt.figure(figsize=(9, 7.5))
gs = gridspec.GridSpec(2, 2, figure=fig)
for i, _f in enumerate(f):
    ghost = fig.add_subplot(gs[i])
    ghost.set_title(f'$f =$ {_f} GHz\n')
    ghost.axis('off')
    gs_in = gs[i].subgridspec(2, 2)
    for i_in, _r in enumerate(r):
        ax = fig.add_subplot(gs_in[i_in])
        ax.plot(h, dataset[(dataset['f']==_f) & (dataset['r']==_r)]['rpd_sph'],
                'ko-', label='spherical', **markerkwargs)
        ax.plot(h, dataset[(dataset['f']==_f) & (dataset['r']==_r)]['rpd_cyl'],
                'kd--', dashes=(3, 1.5), label='cylndrical', **markerkwargs)
        ax.set(title=f'$r =$ {_r} cm', xscale='log')
        if (_f in [60, 90]) & (_r in [10, 15]):
            ax.set(xlabel='', xticks=h, xticklabels=h)
        else:
            ax.set(xlabel='', xticks=h, xticklabels=[])
        if (_f in [6, 60]) & (_r in [5, 10]):
            ax.set(ylabel='', yticks=[0, 10, 20])
        else:
            ax.set(ylabel='', yticks=[0, 10, 20], yticklabels=[])

handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1.225, 0.595))
fig.supxlabel('$d$ [mm]')
fig.supylabel('$sPD_{\\text{n}}$ [W/m$^2$]')
fig.tight_layout()
fname = os.path.join('figures', 'sPDn_rpd')
save_fig(fig, fname=fname)

### RPD between incident power densities on the spherical and planar surface at 26 GHz

In [ ]:
fig_config(latex=True, scaler=1.25, text_size=18)
fig = plt.figure()
heat = dataset[dataset['f']==26].pivot('r', 'd', 'rpd_sph')
ax = sns.heatmap(heat,
                 annot=np.array([f'\\textbf{{{data:.2g}}}'
                                 for data in heat.values.ravel()]
                               ).reshape(heat.shape),
                 fmt='',
                 linewidth=2,
                 cmap='binary',
                 vmin=0, vmax=np.ceil(heat.to_numpy().max()),
                 cbar_kws={'label': 'RPD [\%]', 'ticks': [0, 5, 10, 15]},
                 square=False)
ax.set(xlabel='$d$ [mm]', ylabel='$r$ [cm]', title='spherical surface');
fname = os.path.join('figures', 'heat_sph_rpd')
save_fig(fig, fname=fname, formats=['png'])

### RPD between incident power densities on the cylndrical and planar surface at 26 GHz

In [ ]:
fig_config(latex=True, scaler=1.25, text_size=18)
fig = plt.figure()
heat = dataset[dataset['f']==26].pivot('r', 'd', 'rpd_cyl')
ax = sns.heatmap(heat,
                 annot=np.array([f'\\textbf{{{data:.2g}}}'
                                 for data in heat.values.ravel()]
                               ).reshape(heat.shape),
                 fmt='',
                 linewidth=2,
                 cmap='binary',
                 vmin=0, vmax=np.round(heat.to_numpy().max(), 1),
                 cbar_kws={'label': 'RPD [\%]', 'ticks': [0, 2.2, 4.4]},
                 square=False)
ax.set(xlabel='$d$ [mm]', ylabel='$r$ [cm]', title='cylindrical surface');
fname = os.path.join('figures', 'heat_cyl_rpd')
save_fig(fig, fname=fname, formats=['png'])